# BigQuery Data for TensorFlow Train and Save then AI Platform for Serving

In [1]:
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import tensorflow as tf

In [201]:
PROJECT_ID='statmike-mlops'
BQDATASET_ID='digits'
BQTABLE_ID='digits_prepped'
MODEL_DIR='gs://statmike-models/digits/keras'
BATCH_SIZE = 30

https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas

In [207]:
from google.cloud import bigquery
bqclient = bigquery.Client()
bqjob = bqclient.query(
"""
SELECT * FROM `"""+BQDATASET_ID+""".INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
WHERE TABLE_NAME = '"""+BQTABLE_ID+"""' """
)
schema = bqjob.result().to_dataframe()
schema

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description
0,statmike-mlops,digits,digits_prepped,p0,p0,FLOAT64,None
1,statmike-mlops,digits,digits_prepped,p1,p1,FLOAT64,None
2,statmike-mlops,digits,digits_prepped,p2,p2,FLOAT64,None
3,statmike-mlops,digits,digits_prepped,p3,p3,FLOAT64,None
4,statmike-mlops,digits,digits_prepped,p4,p4,FLOAT64,None
...,...,...,...,...,...,...,...
62,statmike-mlops,digits,digits_prepped,p62,p62,FLOAT64,None
63,statmike-mlops,digits,digits_prepped,p63,p63,FLOAT64,None
64,statmike-mlops,digits,digits_prepped,target,target,INT64,None
65,statmike-mlops,digits,digits_prepped,target_OE,target_OE,STRING,None


In [208]:
OMIT = ['target_OE','SPLITS']

selected_fields=schema[~schema.column_name.isin(OMIT)].column_name.tolist()

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != 'target':
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

from tensorflow.python.framework import dtypes
output_types=schema[~schema.column_name.isin(OMIT)].data_type.tolist()
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

https://www.tensorflow.org/io/api_docs/python/tfio/bigquery/BigQueryClient

In [209]:
def transTable(row_dict):
    target=row_dict.pop('target')
    target = tf.one_hot(tf.cast(target,tf.int64),10)
    target = tf.cast(target,tf.float32)
    return(row_dict,target)

In [210]:
client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TRAIN'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)

In [211]:
train = table.shuffle(100000).batch(BATCH_SIZE)

In [212]:
client = BigQueryClient()
session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TEST'",requested_streams=3)
table = session.parallel_read_rows()
table = table.map(transTable)
test = table.batch(BATCH_SIZE)

In [213]:
for a, b in train.take(1):
    columns=list(a.keys())
    print(columns)
    #print(a)
    print(b)

['p0', 'p1', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p2', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p3', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p4', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p5', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p6', 'p60', 'p61', 'p62', 'p63', 'p7', 'p8', 'p9']
tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.

In [199]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.DenseFeatures(feature_columns))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [217]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)
model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()],outputs=tf.keras.layers.Dense(10,activation=tf.nn.softmax)(feature_layer_outputs))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [218]:
history = model.fit(train,epochs=50)

Epoch 1/50
48/48 [==============================] - 2s 9ms/step - loss: 5.5531 - accuracy: 0.3159
Epoch 2/50
48/48 [==============================] - 1s 10ms/step - loss: 0.6995 - accuracy: 0.8314
Epoch 3/50
48/48 [==============================] - 1s 9ms/step - loss: 0.4056 - accuracy: 0.8725
Epoch 4/50
48/48 [==============================] - 1s 12ms/step - loss: 0.3496 - accuracy: 0.8834
Epoch 5/50
48/48 [==============================] - 1s 11ms/step - loss: 0.2510 - accuracy: 0.9273
Epoch 6/50
48/48 [==============================] - 1s 11ms/step - loss: 0.2168 - accuracy: 0.9334
Epoch 7/50
48/48 [==============================] - 1s 9ms/step - loss: 0.1888 - accuracy: 0.9429
Epoch 8/50
48/48 [==============================] - 1s 10ms/step - loss: 0.1787 - accuracy: 0.9456
Epoch 9/50
48/48 [==============================] - 1s 10ms/step - loss: 0.1485 - accuracy: 0.9559
Epoch 10/50
48/48 [==============================] - 1s 8ms/step - loss: 0.1620 - accuracy: 0.9586
Epoch 11/50
4

In [165]:
loss, accuracy = model.evaluate(test)

Consider rewriting this model with the Functional API.
13/13 [==============================] - 2s 12ms/step - loss: 0.0991 - accuracy: 0.9643


In [167]:
model.predict(test.take(1))

Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.


array([[2.69354677e-15, 1.18419011e-10, 4.67168361e-12, 1.32369338e-09,
        1.74876334e-06, 3.30453486e-06, 7.98585780e-14, 9.99994874e-01,
        3.38779525e-11, 1.10348422e-20],
       [1.63387411e-13, 5.64952751e-09, 1.50661081e-10, 1.18749388e-09,
        2.14398415e-06, 4.19806156e-06, 2.57184833e-12, 9.99993682e-01,
        7.58372753e-10, 3.11227181e-18],
       [5.13627701e-14, 7.38864699e-12, 2.01428783e-12, 2.21441443e-09,
        4.22987512e-07, 1.07488095e-05, 7.22084849e-12, 9.99988794e-01,
        2.38800812e-10, 4.17148051e-18],
       [8.12774985e-08, 3.84291080e-12, 1.20785828e-07, 2.03604986e-06,
        1.56185823e-03, 1.81388453e-01, 8.79865474e-05, 6.84810162e-01,
        1.32149309e-01, 1.07284709e-10],
       [2.35668963e-07, 7.30548468e-15, 1.06654761e-06, 5.84374701e-11,
        1.00592915e-02, 9.41620165e-05, 1.05688869e-07, 2.13835984e-02,
        9.68461573e-01, 3.50983962e-15],
       [1.00496869e-07, 2.12737461e-07, 7.62030482e-03, 1.12163209e-10,
   

In [202]:
model.save(MODEL_DIR)

Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
INFO:tensorflow:Assets written to: gs://statmike-models/digits/keras/assets


# Remove Resources
- undeploy-model
- remove endpoint
- remove model

In [203]:
!gsutil rm -r $MODEL_DIR

Removing gs://statmike-models/digits/keras/#1617663061696426...
Removing gs://statmike-models/digits/keras/assets/#1617663064722256...          
Removing gs://statmike-models/digits/keras/saved_model.pb#1617663065086705...   
Removing gs://statmike-models/digits/keras/variables/#1617663061897932...       
/ [4 objects]                                                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rm ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Removing gs://statmike-models/digits/keras/variables/variables.data-00000-of-00001#1617663063733934...
Removing gs://statmike-models/digits/keras/variables/variables.index#1617663064026656...
/ [6 objects]                                                                   
Operation completed over 6 objects.                                              
